In [ ]:
data_path = "/media/quillaur/1f547fcd-9595-4de6-86d1-7de57051e876/datasets/cat_dogs/"

# Load dataset

https://www.tensorflow.org/tutorials/images/classification

In [1]:
from tensorflow.keras.utils import image_dataset_from_directory

2022-09-14 20:25:35.149022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-14 20:25:35.275194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-14 20:25:35.275209: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-14 20:25:35.299057: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-14 20:25:36.144308: W tensorflow/stream_executor/platform/de

In [ ]:
batch_size = 32
size = (224, 224)

In [ ]:
train_ds, val_ds = image_dataset_from_directory(
  data_path,
  validation_split=0.3,
  subset="both",
  seed=123,
  image_size=size,
  batch_size=batch_size)

# CNN

In [ ]:
from tensorflow import keras

In [ ]:
num_classes = len(train_ds.class_names)

model = keras.Sequential([
  keras.Input(shape=(224, 224, 3)),
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes)])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)
model.summary()

In [ ]:
epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

# Transfert learning

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
model.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = model(inputs)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

model.summary()

In [ ]:
epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=val_ds)